# Model Tests
Each model will be tested against other stocks in the list

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from os import makedirs

## Data Preprocessing

In [2]:
stock_dict = {
    'MEG': [],
    'JGS': [],
    'BDO': [],
    'FGEN': [],
    'ICT': [],
    'ALI': [],
    'SMC': [],
    'TEL': [],
    'GLO': [],
    'BLOOM': [],
    'RLC': [],
    'MER': [],
    'AC': [],
    'PGOLD': [],
    'LTG': [],
    'MPI': [],
    'AP': [],
    'RRHI': [],
    'URC': [],
    'PSEI': []
}

In [3]:
for index, key in enumerate(stock_dict):
    # Loading of datasets to the dictionary of stocks
    # We will only use the closing prices, hence we set usecols to index:4
    data = np.genfromtxt(f'data/{key}.csv', delimiter=',', skip_header=1, usecols=4)

    # If PSEI, we cut the data (include only the test data)
    if key == 'PSEI':
        len_data = len(data)
        data = data[int(len_data*0.80):]
    stock_dict[key] = data

print(stock_dict)

{'MEG': array([2.3408, 2.3145, 2.2882, ..., 2.37  , 2.31  , 2.34  ]), 'JGS': array([ 7.6149,  7.695 ,  7.695 , ..., 54.    , 53.6   , 53.9   ]), 'BDO': array([ 17.8868,  17.8868,  17.8868, ..., 126.    , 125.    , 125.5   ]), 'FGEN': array([19.5961, 19.8114, 19.8114, ..., 18.1   , 18.18  , 18.02  ]), 'ICT': array([  4.9185,   4.9185,   4.6815, ..., 212.    , 211.4   , 214.    ]), 'ALI': array([ 6.9136,  6.9136,  9.3827, ..., 29.2   , 29.1   , 29.25  ]), 'SMC': array([ 45.8774,  45.8774,  43.9855, ..., 112.7   , 110.    , 107.5   ]), 'TEL': array([ 827.5,  827.5,  700. , ..., 1395. , 1380. , 1380. ]), 'GLO': array([ 749.8907,  759.6295,  740.1518, ..., 2090.    , 2080.    ,
       2016.    ]), 'BLOOM': array([0.72, 0.7 , 0.9 , ..., 9.4 , 9.44, 9.45]), 'RLC': array([ 2.9573,  2.9957,  3.3029, ..., 16.42  , 16.3   , 16.14  ]), 'MER': array([ 95.9033,  95.1712,  97.3675, ..., 294.    , 283.    , 315.    ]), 'AC': array([198.2492, 195.6745, 205.9732, ..., 714.    , 700.    , 700.    ]), 'PG

In [4]:
stock_symbols = [keys for index,keys in enumerate(stock_dict)]
print(stock_symbols)

['MEG', 'JGS', 'BDO', 'FGEN', 'ICT', 'ALI', 'SMC', 'TEL', 'GLO', 'BLOOM', 'RLC', 'MER', 'AC', 'PGOLD', 'LTG', 'MPI', 'AP', 'RRHI', 'URC', 'PSEI']


In [5]:
# Creating a windowed datasets for training, using the following window sizes (5, 10, 15, 20)
windowed_sizes = [5, 10, 15, 20]
windowed_data = [0, 0, 0, 0]

# Initialize index counter 
idx_counter = 0

# Loop through the different window sizes
X_stock_dict = stock_dict.copy()
idx_counter = 0
for stock in stock_symbols:
    len_data = X_stock_dict[stock].shape[0]
    for window_size in windowed_sizes:
        train_data = np.zeros((len_data - window_size, window_size))
        for i in range(len_data - window_size):
            train_data[i] = X_stock_dict[stock][i:i + window_size]
        windowed_data[idx_counter] = train_data
        idx_counter += 1

    X_stock_dict[stock] = windowed_data
    # Reset windowed_data and idx_counter
    windowed_data = [0, 0, 0, 0]
    idx_counter = 0

## Predicting each stocks for every model

In [6]:
# Load the model_s5
model = tf.keras.models.load_model('exported_models/model_s5.keras')

# Initialize pandas dataframe for model_s5
len_symbols = len(stock_symbols)
df_model_s5 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_s5
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][0])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 5)')
    plt.title(f"Actual vs Predictions using model_s5 for {stock}")
    plt.legend()
    # Save the graph in ./results/test_validation/graphs
    makedirs('./results/test_validation/graphs/model_s5', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_s5/{stock}.png')
    # Reset the figure
    plt.clf()

    len_predictions = len(predictions)
    df_model_s5.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][5:], predictions)
    df_model_s5.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][5:], predictions))
    df_model_s5.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][5:], predictions)
    df_model_s5.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][5:], predictions)

36/36 [==============================] - 0s 998us/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [7]:
# Save the dataframe in ./results/test_validation/model
makedirs('./results/test_validation/model', exist_ok=True)
df_model_s5.to_csv('./results/test_validation/model/model_s5.csv')

In [8]:
# Load the model_s10
model = tf.keras.models.load_model('exported_models/model_s10.keras')

# Initialize pandas dataframe for model_s10
len_symbols = len(stock_symbols)
df_model_s10 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_s10
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][1])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 10)')
    plt.title(f"Actual vs Predictions using model_s10 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_s10', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_s10/{stock}.png')
    # Reset the figure
    plt.clf()

    len_predictions = len(predictions)
    df_model_s10.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][10:], predictions)
    df_model_s10.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][10:], predictions))
    df_model_s10.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][10:], predictions)
    df_model_s10.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][10:], predictions)

36/36 [==============================] - 0s 910us/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [9]:
# Save the dataframe in ./results/test_validation/model
makedirs('./results/test_validation/model', exist_ok=True)
df_model_s10.to_csv('./results/test_validation/model/model_s10.csv')

In [10]:
# Load the model_s15
model = tf.keras.models.load_model('exported_models/model_s15.keras')

# Initialize pandas dataframe for model_s15
len_symbols = len(stock_symbols)
df_model_s15 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_s15
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][2])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 15)')
    plt.title(f"Actual vs Predictions using model_s15 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_s15', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_s15/{stock}.png')
    plt.clf()

    len_predictions = len(predictions)
    df_model_s15.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][15:], predictions)
    df_model_s15.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][15:], predictions))
    df_model_s15.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][15:], predictions)
    df_model_s15.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][15:], predictions)

35/35 [==============================] - 0s 1ms/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [11]:
# Save the dataframe in ./results/test_validation/model
makedirs('./results/test_validation/model', exist_ok=True)
df_model_s15.to_csv('./results/test_validation/model/model_s15.csv')

In [12]:
# Load the model_s20
model = tf.keras.models.load_model('exported_models/model_s20.keras')

# Initialize pandas dataframe for model_s20
len_symbols = len(stock_symbols)
df_model_s20 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_s20
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][3])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 20)')
    plt.title(f"Actual vs Predictions using model_s20 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_s20', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_s20/{stock}.png')
    plt.clf()

    len_predictions = len(predictions)
    df_model_s20.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][20:], predictions)
    df_model_s20.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][20:], predictions))
    df_model_s20.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][20:], predictions)
    df_model_s20.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][20:], predictions)

35/35 [==============================] - 0s 1ms/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [13]:
# Save the dataframe in ./results/test_validation/model
makedirs('./results/test_validation/model', exist_ok=True)
df_model_s20.to_csv('./results/test_validation/model/model_s20.csv')

In [14]:
# Load the model_baseline5
model = tf.keras.models.load_model('exported_models/model_baseline5.keras')

# Initialize pandas dataframe for model_baseline5
len_symbols = len(stock_symbols)
df_model_baseline5 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_baseline5
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][0])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 5)')
    plt.title(f"Actual vs Predictions using model_baseline5 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_baseline5', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_baseline5/{stock}.png')
    plt.clf()

    len_predictions = len(predictions)
    df_model_baseline5.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][5:], predictions)
    df_model_baseline5.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][5:], predictions))
    df_model_baseline5.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][5:], predictions)
    df_model_baseline5.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][5:], predictions)

36/36 [==============================] - 0s 970us/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [15]:
# Save the dataframe in ./results/test_validation/model
makedirs('./results/test_validation/model', exist_ok=True)
df_model_baseline5.to_csv('./results/test_validation/model/model_baseline5.csv')

In [16]:
# Load the model_baseline10
model = tf.keras.models.load_model('exported_models/model_baseline10.keras')

# Initialize pandas dataframe for model_baseline10
len_symbols = len(stock_symbols)
df_model_baseline10 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_baseline10
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][1])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 10)')
    plt.title(f"Actual vs Predictions using model_baseline10 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_baseline10', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_baseline10/{stock}.png')
    plt.clf()

    len_predictions = len(predictions)
    df_model_baseline10.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][10:], predictions)
    df_model_baseline10.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][10:], predictions))
    df_model_baseline10.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][10:], predictions)
    df_model_baseline10.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][10:], predictions)

36/36 [==============================] - 0s 1ms/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [17]:
makedirs('./results/test_validation/model', exist_ok=True)
df_model_baseline10.to_csv('./results/test_validation/model/model_baseline10.csv')

In [18]:
# Load the model_baseline15
model = tf.keras.models.load_model('exported_models/model_baseline15.keras')

# Initialize pandas dataframe for model_baseline15
len_symbols = len(stock_symbols)
df_model_baseline15 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_baseline15
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][2])
    # Plot each one
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 15)')
    plt.title(f"Actual vs Predictions using model_baseline15 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_baseline15', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_baseline15/{stock}.png')
    plt.clf()

    len_predictions = len(predictions)
    df_model_baseline15.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][15:], predictions)
    df_model_baseline15.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][15:], predictions))
    df_model_baseline15.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][15:], predictions)
    df_model_baseline15.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][15:], predictions)

35/35 [==============================] - 0s 1ms/step


<Figure size 640x480 with 0 Axes>

In [19]:
makedirs('./results/test_validation/model', exist_ok=True)
df_model_baseline15.to_csv('./results/test_validation/model/model_baseline15.csv')

In [20]:
# Load the model_baseline20
model = tf.keras.models.load_model('exported_models/model_baseline20.keras')

# Initialize pandas dataframe for model_baseline20
len_symbols = len(stock_symbols)
df_model_baseline20 = pd.DataFrame({
    'MSE': [0 for _ in range(len_symbols)],
    'RMSE': [0 for _ in range(len_symbols)],
    'MAE': [0 for _ in range(len_symbols)],
    'MAPE': [0 for _ in range(len_symbols)]
}, index=[stock for stock in stock_symbols])

# Predict for all stocks using model_baseline20
for stock in stock_symbols:
    predictions = model.predict(X_stock_dict[stock][3])
    # Plot each one
    plt.figure(figsize=(10, 5))
    plt.plot(stock_dict[stock][0], label=f'Actual for {stock}')
    plt.plot(predictions, label='Predicted (window_size = 20)')
    plt.title(f"Actual vs Predictions using model_baseline20 for {stock}")
    plt.legend()
    makedirs('./results/test_validation/graphs/model_baseline20', exist_ok=True)
    plt.savefig(f'./results/test_validation/graphs/model_baseline20/{stock}.png')
    plt.clf()

    len_predictions = len(predictions)
    df_model_baseline20.loc[stock, 'MSE'] = mean_squared_error(stock_dict[stock][20:], predictions)
    df_model_baseline20.loc[stock, 'RMSE'] = sqrt(mean_squared_error(stock_dict[stock][20:], predictions))
    df_model_baseline20.loc[stock, 'MAE']= mean_absolute_error(stock_dict[stock][20:], predictions)
    df_model_baseline20.loc[stock, 'MAPE'] = mean_absolute_percentage_error(stock_dict[stock][20:], predictions)

35/35 [==============================] - 0s 1ms/step


<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

...

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

In [21]:
makedirs('./results/test_validation/model', exist_ok=True)
df_model_baseline20.to_csv('./results/test_validation/model/model_baseline20.csv')